# Site Dev

# Site App with Database v4

* About Me Section
* Counter
* Portfolio
* Notepad: Create, Read
* Agent + Vertex AI

## Create a database server using Docker

### Postgres Docker

In [ ]:
# Without volume/data connected
# !docker run --name postgres-sql -e POSTGRES_USER=matt -e POSTGRES_PASSWORD=password  -d postgres

In [ ]:
# With volume/data connected
!docker run -d \
    --name postgres-sql \
    -e POSTGRES_USER=matt \
    -e POSTGRES_PASSWORD=password \
    -v $(pwd)/data/:/var/lib/postgresql/data/ \
    -p 5000:5432 \
    postgres

In [ ]:
!docker ps

In [ ]:
!ls -a

In [ ]:
# Check the data
# In a terminal
# sudo su
# cd $(pwd)/data/
# ls -a

In [ ]:
# Test the data of the DB
# docker rm -f <CONTAINER_ID>
# run the docker run <...>
# check the pgAdmin
# the data persists

### pgAdmin

In [ ]:
!docker run -p 8000:80 \
    -e 'PGADMIN_DEFAULT_EMAIL=matt@example.com' \
    -e 'PGADMIN_DEFAULT_PASSWORD=password' \
    -d dpage/pgadmin4

In [ ]:
!docker ps

### Create a firewall (GCP)

In [ ]:
!gcloud compute --project=$(gcloud config get project) firewall-rules create ports \
    --direction=INGRESS --priority=1000 --network=default --action=ALLOW --rules=tcp:5000,tcp:8000,tcp:8501,tcp:9000 --source-ranges=0.0.0.0/0

### Open pgAdmin Browser

In [ ]:
# http://<IP_ADDRESS>:<PORT>
# Login with credentials

# Register Server
# HOST: <IP_ADDRESS>
# PORT: 5000
# USER: <POSTGRES_USER>
# PASSWORD: <PASSWORD>

## Create the app and create and run the image

### Install and import libraries

In [ ]:
!pip install streamlit psycopg2-binary -q

In [ ]:
# Restart the kernel

In [1]:
import streamlit as st
import psycopg2

In [ ]:
!mkdir app

### Application

In [ ]:
cd app

In [ ]:
pwd

In [ ]:
%%writefile env.py
# DB Credentials
DBNAME="matt"
USER='matt'
HOST=''
PORT='5000'
PASSWORD='password'

In [ ]:
#%writefile app.py

In [ ]:
%%writefile Dockerfile
FROM python
WORKDIR /app
RUN pip install streamlit psycopg2-binary -q
COPY app.py env.py ./app/
EXPOSE 8501
CMD ["streamlit", "run", "app.py", "--server.address", "0.0.0.0"]

In [ ]:
cd app

In [ ]:
pwd

### Build the image locally image

In [ ]:
!docker build -t app . -q

In [ ]:
# List images
!docker images

In [ ]:
# Run the app in detached mode
!docker run -d -p 8501:8501 -v $(pwd):/app app

In [ ]:
!docker ps -a

In [ ]:
# Remove all docker ps
# !docker rm -f $(docker ps -aq)

### Connect to the database from Jupyter Lab

In [ ]:
# Import libraries
import psycopg2
import os
import time
from env import *

# Variable
database_name = DBNAME

con = psycopg2.connect(f"""
                       dbname={DBNAME}
                       user={USER}
                       host={HOST}
                       port={PORT}
                       password={PASSWORD}
                       """)
cur = con.cursor()

In [ ]:
cur = con.cursor()
SQL = "SELECT * FROM notes"
cur.execute(SQL)
cur.fetchall()

In [ ]:
dir(st)

In [ ]:
id = 46
name='Matt'
header='Hello'
note='Hi'

In [ ]:
# Update
cur.execute(f"UPDATE notes SET id={id}, name='{name}', header='{header}', note='{note}' WHERE id = {id}")
con.commit()